<a href="https://colab.research.google.com/github/Namsik-Yoon/pytorch_basic/blob/master/2_1_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80_with_My_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

pytorch 기초에서 사용한 boston 데이터셋을 바탕으로 선형회귀 모델 학습

# Data Load

In [0]:
import pandas as pd
from sklearn.datasets import load_boston

boston = load_boston()

In [0]:
boston_df = pd.DataFrame(data=boston['data'],columns=boston['feature_names'])
boston_df

scikit-learn이 제공하는 회귀 분석용 예제 데이터에 대해 소개한다. 먼저, 보스턴 주택 가격 데이터는 다음과 같이 구성되어 있다.



*   타겟 데이터
    * 1978 보스턴 주택 가격
    * 506개 타운의 주택 가격 중앙값 (단위 1,000 달러)
*   특징 데이터
    * CRIM: 범죄율
    * ZN: 25,000 평방피트를 초과 거주지역 비율
    * INDUS: 비소매상업지역 면적 비율
    * CHAS: 찰스강의 경계에 위치한 경우는 1, 아니면 0
    * NOX: 일산화질소 농도
    * RM: 주택당 방 수
    * AGE: 1940년 이전에 건축된 주택의 비율
    * DIS: 직업센터의 거리
    * RAD: 방사형 고속도로까지의 거리
    * TAX: 재산세율
    * B: 인구 중 흑인 비율
    * PTRATIO: 학생/교사 비율
    * LSTAT: 인구 중 하위 계층 비율



In [0]:
data = boston_df
data = data.apply(
    lambda x: (x - x.mean()) / x.std()
)
data['Price'] = boston['target']
data

In [0]:
X,y = data.values[:,:-1],data.values[:,-1:]
print(X.shape,y.shape)

# Set Custom Dataset

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [0]:
class MyDataset(Dataset):
    def __init__(self):
        self.x_data = torch.tensor(X,dtype=torch.float)
        self.y_data = torch.tensor(y,dtype=torch.float)
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self,idx):
        x = self.x_data[idx]
        y = self.y_data[idx]
        return x,y

In [0]:
dataset = MyDataset()
dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=True)

In [0]:
model = nn.Linear(13,1)
optimizer = optim.SGD(model.parameters(), lr=0.05)

In [0]:
nb_epochs = 50
for epoch in range(nb_epochs + 1):
    for samples in dataloader:
        # print(batch_idx)
        # print(samples)
        x_train, y_train = samples
        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs,cost.item()))

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [0]:
reg_model = LinearRegression()
reg_model.fit(boston['data'],boston['target'])
pred_y = reg_model.predict(boston['data'])

In [0]:
mean_squared_error(boston['target'],pred_y )